## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 24
country = "Denmark"; # Possible options: Portugal, Slovenia, Greece, Denmark, this if apply only to Slovenia and Denmark, because both countries have biomass

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 17:43:40


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.Pl = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.Php = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,3]))
model.Pb = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,4]))
model.Pimp = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,5]))
model.Ppv = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,6]))
model.Pwind = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,7]))
model.Phid = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,8]))
model.Pbio = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,9]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,9]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 0.95
model.m = 0.6
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.PEVdc = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Pgen = pyo.Var(model.t, domain =  pyo.NonNegativeReals, initialize = 0)
model.Pres = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Pex = pyo.Var(model.t, domain = pyo.NonNegativeReals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] >= m.EEVmax[ev]*m.m
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n  - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _RES_balance(m,t):
    return m.Pres[t] == m.Pwind[t] + m.Ppv[t] + m.Phid[t] + m.Pbio[t]
model.RES_balance = pyo.Constraint(model.t, rule = _RES_balance)

def _total_balance(m,t):
    #if (country == "Slovenia"):
        return m.Pgen[t] + m.Pres[t] + m.Pimp[t] - m.Pex[t] == m.Pl[t] + m.Pb[t] +sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
    #else:
        #return m.Pgen[t] + m.Pres[t] + m.Pimp[t] - m.Pex[t] ==  m.Pl[t] + m.Pb[t] + sum(m.PEV[ev, t] for ev in np.arange(1, n_evs + 1))
model.total_balance = pyo.Constraint(model.t, rule = _total_balance)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _power_discharging_limit2(m,ev,t): 
    return m.PEVdc[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_discharging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_discharging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
    #return m.EEVmin[ev] <= m.EEV[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)


## Objective function

In [11]:
def _FOag(m):
    return (sum(m.Pgen[t] for t in np.arange(1, n_time + 1))**2 + sum(m.Ppv[t] for t in np.arange(1, n_time + 1))**2)/ (sum(m.Ppv[t] for t in np.arange(1, n_time + 1))**2) + sum([m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2 for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmp1hpwx9kj
  Lower bound: 588287586.7011604
  Upper bound: 588287586.7011604
  Number of objectives: 1
  Number of constraints: 1069650
  Number of variables: 911449
  Number of nonzeros: 2126641
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 5.63
  Termination condition: optimal
  Termination message: Barrier - Optimal\x3a Objective = 5.8828758670e+08
  Error rc: 0
  Time: 13.17224645614624
# ----------------------------------------------------------
#   Solution Information
# ------------------

## Objective Function Value

In [13]:
pyo.value(model.FOag)

588287586.7010498

## End Time

In [14]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 17:48:17
Dif: 0:04:37


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
EEV_df = ext_pyomo_vals(model.EEV)
PEVdc_df = ext_pyomo_vals(model.PEVdc)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))

Charge cost: 18510023.285770793


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy()/1000000)))

Total Charge: 103.19759295060352


In [18]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,64680.000000,...,62678.846196,63342.460906,64018.230129,64702.859623,65386.532077,66064.851865,66730.087191,67375.366235,67993.619058,65280.543063
2,38566.513705,38566.513705,38566.513705,38566.513705,38566.513705,38566.513705,38566.513705,38566.513705,38566.513705,38566.513705,...,36064.264687,36676.143701,37296.231665,37920.773745,38540.610692,39151.609771,39746.324567,40317.730538,40858.805934,41390.156886
3,8537.885273,8436.965804,8335.915523,8243.948352,8165.342037,8102.402299,8057.464232,8030.430532,8021.788334,8032.057890,...,8360.598293,8360.598293,8360.598293,8360.598293,8360.598293,8360.598293,8360.598293,8360.598293,8360.598293,8514.305532
4,8537.991324,8437.478730,8336.850822,8245.348323,8167.273281,8104.955446,8060.757431,8034.605520,8027.015906,8038.537880,...,8366.015109,8366.015109,8366.015109,8366.015109,8366.015109,8366.015109,8366.015109,8366.015109,8366.015109,8512.540802
5,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,...,39862.563330,39421.020728,41063.035548,42751.776926,44474.924406,44194.505799,43921.780750,43626.584255,43259.172719,41226.131241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6325,45963.696894,47343.723015,48753.081678,50187.391337,51654.957689,53166.450769,54739.449140,56390.687241,58139.850533,60023.262600,...,62637.334961,58500.071277,54362.807593,50225.543908,50225.543908,46088.280224,41951.016539,41951.016539,37813.752855,44726.124310
6326,47362.676495,47466.003018,47570.038670,47674.979481,47780.992943,47888.180355,47998.180053,48109.418407,48222.098821,48336.384957,...,48006.987786,48006.987786,48006.987786,48006.987786,48006.987786,48006.987786,48006.987786,48006.987786,48006.987786,47320.777772
6327,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,57000.000000,...,55606.408169,56117.991425,56636.664113,57160.103693,57679.745977,58191.827895,58689.517236,59166.894483,59618.780984,57641.913020
6328,65923.585108,66842.806937,67770.113654,68706.568101,69652.802237,70609.082678,71584.605638,72569.677383,73567.263916,74578.249782,...,69761.380378,69761.380378,69761.380378,69761.380378,69761.380378,69761.380378,69761.380378,69761.380378,69761.380378,65040.793115


In [19]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1279.408719,1285.188807,1302.441871,1316.051717,1318.800549,1315.980802,1303.479322,1281.264901,1247.615630,1228.640539
2,927.277945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1236.141256,1239.217024,1252.691883,1262.965178,1263.832308,1260.156398,1247.774375,1225.775998,1191.181295,1166.259378
3,460.931866,490.601926,496.075157,500.986367,506.565135,513.153065,521.698377,530.143910,539.150254,548.831407,...,600.105435,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,573.640133
4,461.043499,490.746156,496.240903,501.182732,506.802246,513.442412,522.055843,530.581724,539.687111,549.487224,...,601.658899,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,576.398289
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2080.154198,2109.282386,2167.722497,2223.356393,2266.817179,2305.032964,2317.582254,2294.164853,2210.752208,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6325,1645.996731,1771.648798,1815.995322,1855.296834,1903.167436,1962.312167,2039.796610,2134.719941,2249.477269,2401.051915,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11959.697225
6326,592.291048,596.586814,597.786192,598.967373,600.096606,601.109663,603.631270,604.296138,604.978773,605.644002,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,583.164673
6327,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1165.814306,1169.364768,1182.532013,1192.916481,1193.628113,1189.323318,1176.123602,1153.863406,1121.330028,1100.869629
6328,1309.036955,1336.448457,1345.901057,1356.389846,1367.433231,1378.612033,1399.283106,1409.529301,1422.615656,1436.294699,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1535.364569


In [20]:
PEV_df.sum()

1     4.158851e+06
2     3.551494e+06
3     3.602022e+06
4     3.651410e+06
5     3.705090e+06
6     3.789901e+06
7     3.886790e+06
8     3.977348e+06
9     4.084831e+06
10    4.246862e+06
11    3.610221e+06
12    3.061824e+06
13    3.292059e+06
14    5.209613e+06
15    4.915261e+06
16    4.218604e+06
17    4.285387e+06
18    4.345488e+06
19    4.375722e+06
20    4.390540e+06
21    4.372035e+06
22    4.313635e+06
23    6.213850e+06
24    7.938757e+06
dtype: float64

In [21]:
Pgen_df = ext_pyomo_vals(model.Pgen)

Pgen_df

,0
1,9.179633e+06
2,8.593682e+06
3,8.417791e+06
4,8.197555e+06
5,8.134697e+06
6,8.542084e+06
7,4.736875e+06
8,4.832502e+06
9,5.112411e+06
10,5.275865e+06


In [22]:
Pres_df = ext_pyomo_vals(model.Pres)

Pres_df

,0
1,5.202029e+06
2,5.864105e+06
3,5.383119e+06
4,4.117823e+06
5,3.434997e+06
6,4.004192e+06
7,5.191122e+06
8,4.282210e+06
9,4.371468e+06
10,3.906558e+06


In [23]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [24]:
Etriprelax_df.sum()

1          0.000005
2          0.000008
3          0.000008
4          0.000008
5          0.000008
6          0.000008
7          0.000008
8          0.000008
9          0.000008
10         0.000008
11         0.000008
12         0.000008
13      1069.988219
14      6531.044290
15      8373.544718
16     12310.295158
17     13701.172069
18     14278.664832
19     12594.527074
20     13224.317838
21     12248.259830
22      8720.518498
23      4694.228238
24    480151.085592
dtype: float64

In [25]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [26]:
Eminsocrelax_df.sum()

1       0.000000
2       0.000005
3       0.000008
4       0.000008
5       0.000008
6       0.000008
7       0.000008
8       0.000008
9       0.000008
10      0.000008
11      0.000008
12      0.000008
13      0.000008
14      0.000008
15      0.000008
16      0.000008
17      0.000008
18      0.000008
19      0.000008
20      0.000008
21      0.000008
22    389.938384
23      0.000008
24      0.000008
dtype: float64

In [27]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [28]:
Etripn_df.sum()

1     3.012639e+06
2     1.166828e+05
3     5.611484e+04
4     1.747803e+04
5     7.542035e+03
6     1.218573e+04
7     1.022651e+04
8     1.182157e+04
9     0.000000e+00
10    1.878932e+06
11    4.450396e+06
12    0.000000e+00
13    2.402219e+06
14    9.198751e+06
15    4.185460e+06
16    4.260775e+06
17    1.203396e+06
18    2.595200e+06
19    7.924992e+05
20    2.940490e+06
21    3.307096e+06
22    2.922840e+06
23    3.333886e+06
24    1.150694e+07
dtype: float64

In [29]:
Pex_df = ext_pyomo_vals(model.Pex)

In [30]:
Pex_df

,0
1,8126.027728
2,6507.679889
3,8237.569636
4,8524.999735
5,8326.963576
6,8487.044091
7,5636.174085
8,10366.553169
9,9118.126107
10,6588.250695


In [31]:
Pbio_df = ext_pyomo_vals(model.Pbio)

In [32]:
Pbio_df

,0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0
10,0.0


## Save results in csv files

In [33]:
import os 
folder = 'results_RES_c_V2G' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')
PEVdc_df.sum().to_csv(folder + '/PEVdc_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')

Pgen_df.to_csv(folder + '/Pgen.csv')
Pres_df.to_csv(folder + '/Pres.csv')
Pex_df.to_csv(folder + '/Pex.csv')